In [1]:
import numpy as np

import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

import sklearn
import sklearn.preprocessing

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import time
import random
import webbrowser

In [2]:
# twitter dataset

with open("ovchinnikov-rutwitterdataset/A.pkl", "rb") as file:
    A = np.load(file, encoding="latin1")

with open("ovchinnikov-rutwitterdataset/labeled_nodes.pkl", "rb") as file:
    labels = np.load(file, encoding="latin1")

with open("ovchinnikov-rutwitterdataset/i2t.pkl", "rb") as file:
    i2t = pickle.load(file)

with open("ovchinnikov-rutwitterdataset/t2i.pkl", "rb") as file:
    t2i = pickle.load(file)

In [3]:
# preprocessing

F = sklearn.preprocessing.normalize(A, axis=1, norm='l1')
B = sklearn.preprocessing.normalize(A.T, axis=1, norm='l1')
d = np.array(labels.todense())[:, 0]

Fcsc = F.tocsc()
Bcsc = B.tocsc()

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int8 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int8 was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)


In [4]:
# data for cross-validation

n_n_elements = d[d < 0].shape[0]
n_p_elements = d[d > 0].shape[0]

print("Spam nodes:\t{}".format(n_n_elements))
print("Not spam nodes:\t{}".format(n_p_elements))

n_indices = random.sample(np.where(d < 0)[0].tolist(), int(n_n_elements * .20 // 1))  # random indices of 20% negative
p_indices = random.sample(np.where(d > 0)[0].tolist(), int(n_p_elements * .20 // 1))  # random indices of 20% positive

d_CV = d.copy()
d_CV[n_indices] = 0
d_CV[p_indices] = 0

print("Spam nodes (80%):\t{}".format(d_CV[d_CV < 0].shape[0]))
print("Not spam nodes (80%):\t{}".format(d_CV[d_CV > 0].shape[0]))

def CV(x):
    return (sum(x[p_indices] > 0) + sum(x[n_indices] < 0)) / (len(p_indices) + len(n_indices))

Spam nodes:	2749
Not spam nodes:	375
Spam nodes (80%):	2200
Not spam nodes (80%):	300


In [5]:
a1 = 0.7
a2 = 0.7
a3 = 0.7

In [6]:
# we are looking for solution x == T(x)

def T(F, B, d, x):
    return a1 * F.dot(x.clip(0)) + a2 * B.dot(x.clip(-np.inf, 0)) + a3 * d

In [7]:
def h(x):
    ans = np.zeros_like(x)
    ans[x > 0.0] = 1.0
    return ans

In [8]:
def obj(F, B, d, x):
    return np.linalg.norm(x - T(F, B, d, x)) ** 2 / 2

In [9]:
def Jac(F, B, d, x):
    return sp.sparse.eye(F.shape[0], format='csr') - a1 * F.multiply(sp.sparse.csc_matrix(h(x).reshape(-1, 1))) - a2 * B.multiply(sp.sparse.csc_matrix(h(-x).reshape(-1, 1)))

In [10]:
def der(F, B, d, x):
    l = x - T(F, B, d, x)
    return Jac(F, B, d, x).T.dot(l)

In [11]:
def Hess(F, B, d, x):
    def matvec(x):
        return matvec.Jt.dot(matvec.J.dot(x))
    matvec.Jt = Jac(F, B, d, x).T.tocsr()
    matvec.J = Jac(F, B, d, x).tocsr()
    return sp.sparse.linalg.LinearOperator(F.shape, matvec)

In [12]:
def LineSearch(F, B, d, x, dx):
    alpha = 1.0
    while alpha + 1.0 != 1.0:
        ans = x + alpha * dx
        if obj(F, B, d, ans) < obj(F, B, d, x):
            return ans
        else:
            alpha /= 2
    raise Exception("Machine precision achieved!")

In [13]:
# Newton

def Newton(F, B, d, maxiter=100, x0=None, tol=1e-5, callback=None):
    if x0 is None:
        x_prev = d.copy()
    else:
        x_prev = x0.copy()

    for k in range(maxiter):
        dx, info = sp.sparse.linalg.cg(Hess(F, B, d, x_prev), -der(F, B, d, x_prev))
        x_next = LineSearch(F, B, d, x_prev, dx)
        if callback is not None:
            callback(k + 1, x_next)
        if np.linalg.norm(der(F, B, d, x_next)) < tol:
            break
        x_prev = x_next.copy()

    return (k + 1), x_next.copy()

In [14]:
file = open("callback.txt", "w")
file.write("Iter\tErr\tCV\tDer\n")

def cbk(k, x):
    file.write("{}\t{}\t{}\t{}\n".format(str(k), str(np.linalg.norm(x - T(F, B, d_CV, x))), str(CV(x)), str(np.linalg.norm(der(F, B, d_CV, x)))))

In [15]:
k, ans = Newton(F, B, d_CV, tol=1e-2, callback=cbk)

In [16]:
file.close()

In [17]:
np.save("1", ans)

In [ ]:
# RepRank via Newton answer

k, ans = Newton(F, B, d, tol=1e-2)
print("Required numeber of iterations for original RepRank: {}.".format(str(k)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][:3].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][-3:].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

In [21]:
# RepRank

def RepRank(F, B, d, maxiter=200, x0=None, tol=1e-8, callback=None):
    if x0 is None:
        x_prev = d.copy()
    else:
        x_prev = x0.copy()

    for k in range(maxiter):
        x_next = T(F, B, d, x_prev)
        n = np.linalg.norm(x_next - x_prev)
        if callback is not None:
            callback(x_next, n)
        if n < tol:
            break
        x_prev = x_next

    ans = x_next.copy()
    return(k + 1, ans.reshape(-1,))

In [22]:
# original RepRank answer

k, ans = RepRank(F, B, d)
print("Required numeber of iterations for original RepRank: {}.".format(str(k)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][:3].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))

for i in list(map(i2t.get, ans.argsort(axis=0)[::-1][-3:].tolist())):
    webbrowser.open_new_tab("https://twitter.com/intent/user?user_id={}".format(str(i)))